# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Loading csv file exported from WeatherPy to new DataFrame:
path = "../output_data/clean_city_data.csv"

# Reading in the csv file and creating the new dataframe:
new_city_df = pd.read_csv(path)

# Cleaning new cihty dataframe:
city_df_cleaned = new_city_df.dropna()
city_df_cleaned


FileNotFoundError: [Errno 2] No such file or directory: '../output_data/clean_city_data.csv'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps:
gmaps.configure(api_key=g_key)

In [ ]:
# Using Lat and Lng as locations:
locations = city_df_cleaned[["Lat", "Lng"]]

# Using Humidity as the weight: 
humidity = city_df_cleaned["Humidity"].astype(float)

In [ ]:
# Add Heatmap layer to map:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig 

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# Narrow down the cities to fit weather conditions"

# Zero cloudiness:
cloud_df = city_df_cleaned[city_df_cleaned["Cloudiness"] == 0]

# A max temperature lower than 80 degrees but higher than 70:
temp_df = cloud_df[(cloud_df["Max Temp"] > 70) & (cloud_df["Max Temp"] < 80)]

# Wind speed less than 10 mph:
wind_df = temp_df[temp_df["Wind Speed"] < 10]

# Drop any rows will null values:
wind_df = wind_df.dropna()
wind_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Store into variable named hotel_df:
hotel_df = wind_df

# Add a "Hotel Name" column to the DataFrame: (setting new columns to hold values)
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters:

# Set params:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
    except: 
        print('Issues with finding a hotel at', row['City'])
        hotel_df.loc[index, "Hotel Name"] = "NA"
    
hotel_df  

In [ ]:
#deletes any rows without hotels in 5000 meters
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NA"]
hotel_df

In [ ]:
# Checking hotels:
len(hotel_df[["Lat", "Lng"]])

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig
